In [87]:
import pandas as pd
from implicit.nearest_neighbours import CosineRecommender
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [88]:
df = pd.read_csv("rating2.csv")
df.head()

,users,movies,ratings
0,Lisa Rose,Lady in the Water,2.5
1,Lisa Rose,Snakes on a Plane,3.5
2,Lisa Rose,Just My Luck,3.0
3,Lisa Rose,Superman Returns,3.5
4,Lisa Rose,"You, Me and Dupree",2.5


In [89]:
utility = df.pivot(columns="movies", index="users", values="ratings")
utility

movies,Just My Luck,Lady in the Water,Snakes on a Plane,Superman Returns,The Night Listener,"You, Me and Dupree"
users,,,,,,
Claudia Puig,3.0,NaN,3.5,4.0,4.5,2.5
Gene Seymour,1.5,3.0,3.5,5.0,3.0,3.5
Jack Matthews,NaN,3.0,4.0,5.0,3.0,3.5
Lisa Rose,3.0,2.5,3.5,3.5,3.0,2.5
Michael Phillips,NaN,2.5,3.0,3.5,4.0,NaN
Mick LaSalle,2.0,3.0,4.0,3.0,3.0,2.0
Toby,NaN,NaN,4.5,4.0,NaN,1.0


In [90]:
claudia = utility.iloc[0].fillna(0).values
gene = utility.iloc[1].fillna(0).values
cosine_similarity([claudia], [gene])

array([[0.8847892]])

In [91]:
users_to_users = pd.DataFrame(cosine_similarity(utility.fillna(0)))
users_to_users

,0,1,2,3,4,5,6
0,1.000000,0.884789,0.834622,0.928841,0.807057,0.885658,0.702841
1,0.884789,1.000000,0.982325,0.960646,0.862337,0.955691,0.770024
2,0.834622,0.982325,1.000000,0.906551,0.880155,0.928989,0.805550
3,0.928841,0.960646,0.906551,1.000000,0.838220,0.981831,0.712501
4,0.807057,0.862337,0.880155,0.838220,1.000000,0.891702,0.683164
5,0.885658,0.955691,0.928989,0.981831,0.891702,1.000000,0.734179
6,0.702841,0.770024,0.805550,0.712501,0.683164,0.734179,1.000000


In [92]:
toby = 6
scores = zip(users_to_users.loc[toby], utility.index)
sorted(list(scores), key=lambda x: x[0], reverse=True)

[(1.0000000000000002, 'Toby'),
 (0.8055500405148401, 'Jack Matthews'),
 (0.7700242750941049, 'Gene Seymour'),
 (0.7341786512018111, 'Mick LaSalle'),
 (0.712500602099871, 'Lisa Rose'),
 (0.7028414122660656, 'Claudia Puig'),
 (0.6831639278011596, 'Michael Phillips')]

In [96]:
# 1 - cosine_similarity
cosine_distances([claudia], [gene])

array([[0.1152108]])

In [97]:
users_to_users = pd.DataFrame(cosine_distances(utility.fillna(0)))
users_to_users

,0,1,2,3,4,5,6
0,0.000000,0.115211,0.165378,0.071159,0.192943,0.114342,0.297159
1,0.115211,0.000000,0.017675,0.039354,0.137663,0.044309,0.229976
2,0.165378,0.017675,0.000000,0.093449,0.119845,0.071011,0.194450
3,0.071159,0.039354,0.093449,0.000000,0.161780,0.018169,0.287499
4,0.192943,0.137663,0.119845,0.161780,0.000000,0.108298,0.316836
5,0.114342,0.044309,0.071011,0.018169,0.108298,0.000000,0.265821
6,0.297159,0.229976,0.194450,0.287499,0.316836,0.265821,0.000000


In [98]:
toby = 6
scores = zip(users_to_users.loc[toby], utility.index)
sorted(list(scores), key=lambda x: x[0], reverse=True)

[(0.31683607219884036, 'Michael Phillips'),
 (0.29715858773393444, 'Claudia Puig'),
 (0.287499397900129, 'Lisa Rose'),
 (0.26582134879818886, 'Mick LaSalle'),
 (0.22997572490589513, 'Gene Seymour'),
 (0.19444995948515986, 'Jack Matthews'),
 (0.0, 'Toby')]

In [32]:
df["users"] = df["users"].astype("category")
df["movies"] = df["movies"].astype("category")

users = df.users.cat.categories
movies = df.movies.cat.categories

df.head()

,users,movies,ratings
0,Lisa Rose,Lady in the Water,2.5
1,Lisa Rose,Snakes on a Plane,3.5
2,Lisa Rose,Just My Luck,3.0
3,Lisa Rose,Superman Returns,3.5
4,Lisa Rose,"You, Me and Dupree",2.5


In [33]:
ratings = coo_matrix(
    (df["ratings"].astype(float), (df["users"].cat.codes, df["movies"].cat.codes))
).tocsr()
ratings

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 35 stored elements and shape (7, 6)>

In [34]:
from implicit.cpu.als import AlternatingLeastSquares

model = AlternatingLeastSquares()
model.fit(ratings)

  0%|          | 0/15 [00:00<?, ?it/s]

In [35]:
toby = 6

# recommend items for user.
ids, scores = model.recommend(toby, ratings[toby], N=5)
ids, scores

(array([4, 0, 1, 3, 2], dtype=int32),
 array([ 1.3303235e-03, -3.3636764e-03, -4.3621659e-03, -3.4028235e+38,
        -3.4028235e+38], dtype=float32))

In [36]:
movies[ids]

Index(['The Night Listener', 'Just My Luck', 'Lady in the Water',
       'Superman Returns', 'Snakes on a Plane'],
      dtype='object')

In [37]:
superman = 3

# find related items.
ids, scores = model.similar_items(superman, N=5)
ids, scores

(array([3, 2, 5, 4, 1], dtype=int32),
 array([0.9999999 , 0.98929787, 0.19689831, 0.06118698, 0.04498659],
       dtype=float32))

In [13]:
movies[ids]

Index(['Superman Returns', 'Snakes on a Plane', 'You, Me and Dupree',
       'The Night Listener', 'Lady in the Water'],
      dtype='object')

In [14]:
# find related users.
ids, scores = model.similar_users(toby, N=5)
ids, scores

(array([6, 2, 0, 1, 5], dtype=int32),
 array([1.        , 0.78004295, 0.7573654 , 0.7568415 , 0.75485915],
       dtype=float32))

In [19]:
users[ids]

Index(['Michael Phillips', 'Gene Seymour', 'Mick LaSalle', 'Lisa Rose',
       'Jack Matthews'],
      dtype='object')

## Cosine Recommender

In [81]:
model = CosineRecommender()
model.fit(ratings)

/Users/alextanhongpin/Documents/go/recommender-learn/.venv/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 9.226799011230469e-05 seconds
  warnings.warn(


  0%|          | 0/6 [00:00<?, ?it/s]

In [111]:
toby = 6
ids, scores = model.recommend(
    toby, ratings[toby], N=10, filter_already_liked_items=True
)
ids, scores

(array([4, 1, 0, 5, 3, 2], dtype=int32),
 array([8.28976852, 7.83288013, 6.64235501, 0.        , 0.        ,
        0.        ]))

In [113]:
for row in zip(ids, movies[ids], scores):
    print(row)

(np.int32(4), 'The Night Listener', np.float64(8.289768523180454))
(np.int32(1), 'Lady in the Water', np.float64(7.832880132200684))
(np.int32(0), 'Just My Luck', np.float64(6.642355005524493))
(np.int32(5), 'You, Me and Dupree', np.float64(0.0))
(np.int32(3), 'Superman Returns', np.float64(0.0))
(np.int32(2), 'Snakes on a Plane', np.float64(0.0))


In [118]:
ids, scores = model.similar_items(superman)
ids, scores

(array([3, 2, 5, 4, 1, 0], dtype=int32),
 array([1.        , 0.97987806, 0.9153023 , 0.89217015, 0.83648645,
        0.68022977]))

In [119]:
movies[ids]

Index(['Superman Returns', 'Snakes on a Plane', 'You, Me and Dupree',
       'The Night Listener', 'Lady in the Water', 'Just My Luck'],
      dtype='object')

In [120]:
model = CosineRecommender()
model.fit(ratings.T)

/Users/alextanhongpin/Documents/go/recommender-learn/.venv/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.00012803077697753906 seconds
  warnings.warn(


  0%|          | 0/7 [00:00<?, ?it/s]

In [121]:
ids, scores = model.similar_items(toby)
ids, scores

(array([6, 2, 1, 5, 3, 0, 4], dtype=int32),
 array([1.        , 0.80555004, 0.77002428, 0.73417865, 0.7125006 ,
        0.70284141, 0.68316393]))

In [122]:
users[ids]

Index(['Toby', 'Jack Matthews', 'Gene Seymour', 'Mick LaSalle', 'Lisa Rose',
       'Claudia Puig', 'Michael Phillips'],
      dtype='object')

In [ ]:
# [
#     (1.0000000000000002, "Toby"),
#     (0.8055500405148401, "Jack Matthews"),
#     (0.7700242750941049, "Gene Seymour"),
#     (0.7341786512018111, "Mick LaSalle"),
#     (0.712500602099871, "Lisa Rose"),
#     (0.7028414122660656, "Claudia Puig"),
#     (0.6831639278011596, "Michael Phillips"),
# ]